In [57]:
import pandas as pd
import numpy as np
import json

In [58]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20

This notebook calculates unit-values (value per kg) for each CPC activity per exporter–importer pair, then aggregates per exporter–product to derive:
- **unit_value** = total value ÷ total quantity (sum)
- **mean_unit** = mean(unit_value across importers)
- **min_unit** = min(unit_value across importers)
- **max_unit** = max(unit_value across importers)


# 1. Load data

In [59]:
country_codes_fp = 'country_codes_V202501.csv'
cpc_hs_fp = 'cpc_hs_mapping.xlsx'
baci_fp = 'BACI_HS22_Y2023_V202501.csv'

In [60]:
country_codes = pd.read_csv(country_codes_fp)
country_codes

,country_code,country_name,country_iso2,country_iso3
0,4,Afghanistan,AF,AFG
1,8,Albania,AL,ALB
2,12,Algeria,DZ,DZA
3,16,American Samoa,AS,ASM
4,20,Andorra,AD,AND
...,...,...,...,...
233,876,Wallis and Futuna Isds,WF,WLF
234,882,Samoa,WS,WSM
235,887,Yemen,YE,YEM
236,891,Serbia and Montenegro (...2005),CS,SCG


In [61]:
cpc_hs = pd.read_excel(cpc_hs_fp, dtype={'CPC': str, 'HS': str})
cpc_hs

,CPC,CPC_description,HS,HS_description
0,1111,"Wheat, seed",100191,"Cereals: wheat and meslin, other than durum wh..."
1,1112,"Wheat, other",100199,"Cereals: wheat and meslin, other than durum wh..."
2,1121,"Maize (corn), seed",100510,"Cereals: maize (corn), seed"
3,1122,"Maize (corn), other",100590,"Cereals: maize (corn), other than seed"
4,113,Rice,100640,"Cereals: rice, broken"
...,...,...,...,...
655,9441,Site remediation and clean-up services,999999,Commodities not specified according to kind
656,971,"Washing, cleaning and dyeing services",999999,Commodities not specified according to kind
657,97110,Coin-operated laundry services,999999,Commodities not specified according to kind
658,9715,Dyeing and colouring services,999999,Commodities not specified according to kind


In [62]:
baci = pd.read_csv(baci_fp, dtype={'i': int, 'j': int, 'k': str, 'v': float, 'q': float})
baci

,t,i,j,k,v,q
0,2023,4,20,200290,8.013,5.830
1,2023,4,31,252620,9.808,77.150
2,2023,4,31,680221,33.788,76.655
3,2023,4,31,761510,6.604,2.368
4,2023,4,31,860900,1.500,2.200
...,...,...,...,...,...,...
11232734,2023,894,854,903180,8.144,0.101
11232735,2023,894,858,240120,619.897,99.000
11232736,2023,894,858,630900,0.981,0.446
11232737,2023,894,858,820840,4.743,0.191


# 2. Compute unit value for BACI data

In [63]:
# Sum total value and quantity per exporter and HS
grouped = (
    baci
    .groupby(['i', 'j', 'k'], as_index=False)
    .agg(v_sum=('v', 'sum'), q_sum=('q', 'sum'))
)

In [64]:
# Filter out zero or missing quantity to avoid division by zero
grouped = grouped[grouped['q_sum'] > 0].copy()

In [65]:
grouped['unit_value'] = grouped['v_sum'] / grouped['q_sum']
grouped

,i,j,k,v_sum,q_sum,unit_value
0,4,20,200290,8.013,5.830,1.374443
1,4,31,252620,9.808,77.150,0.127129
2,4,31,680221,33.788,76.655,0.440780
3,4,31,761510,6.604,2.368,2.788851
4,4,31,860900,1.500,2.200,0.681818
...,...,...,...,...,...,...
11232734,894,854,903180,8.144,0.101,80.633663
11232735,894,858,240120,619.897,99.000,6.261586
11232736,894,858,630900,0.981,0.446,2.199552
11232737,894,858,820840,4.743,0.191,24.832461


In [66]:
grouped_stats = (
    grouped
    .groupby(['i','k'], as_index=False)
    .agg(
        v_sum=('v_sum', 'sum'),
        q_sum=('q_sum', 'sum'),
        mean_unit=('unit_value', 'mean'),
        min_unit=('unit_value', 'min'),
        max_unit=('unit_value', 'max')
    )
)
grouped_stats['unit_value'] = grouped_stats['v_sum'] / grouped_stats['q_sum']
grouped_stats

,i,k,v_sum,q_sum,mean_unit,min_unit,max_unit,unit_value
0,4,010221,15.775,2.600,6.067308,6.067308,6.067308,6.067308
1,4,010619,5.157,2.060,17.577805,2.355610,32.800000,2.503398
2,4,010641,126.005,15.060,8.366866,8.366866,8.366866,8.366866
3,4,020120,1.000,0.216,4.629630,4.629630,4.629630,4.629630
4,4,020130,2.000,0.465,4.301075,4.301075,4.301075,4.301075
...,...,...,...,...,...,...,...,...
561475,894,970510,165.945,3.619,44.974351,4.211864,236.750000,45.853827
561476,894,970522,406.675,13.946,31.225558,1.538462,124.468813,29.160691
561477,894,970529,290.200,6.933,72.624637,2.930769,459.062500,41.857782
561478,894,970539,37.631,1.656,4391.362725,8.754843,8164.000000,22.724034


# 3. Merge CPC-HS mapping with the BACI data

In [67]:
cpc_hs['HS'] = cpc_hs['HS'].astype(str)
grouped_stats['k'] = grouped_stats['k'].astype(str)

In [68]:
grouped_stats = grouped_stats.merge(
    cpc_hs[['CPC', 'HS']],
    left_on='k',
    right_on='HS',
    how='inner'
)
grouped_stats

,i,k,v_sum,q_sum,mean_unit,min_unit,max_unit,unit_value,CPC,HS
0,4,100310,348.522,1829.000,0.190553,0.190553,0.190553,0.190553,1151,100310
1,4,100390,296.180,1283.000,0.230850,0.230850,0.230850,0.230850,1152,100390
2,4,100410,883.618,1893.000,0.466782,0.466782,0.466782,0.466782,1171,100410
3,4,100590,52.248,106.000,0.312000,0.124000,0.500000,0.492906,1122,100590
4,4,100640,9.920,24.800,0.400000,0.400000,0.400000,0.400000,113,100640
...,...,...,...,...,...,...,...,...,...,...
51153,894,940370,3.738,5.831,0.789230,0.078065,1.523084,0.641056,38140,940370
51154,894,940410,38.358,57.964,1.790684,0.403077,6.045833,0.661756,3815,940410
51155,894,940610,66.817,45.378,1.339327,0.267841,2.410812,1.472454,3870,940610
51156,894,940690,1495.198,669.982,6.809968,1.569955,11.635500,2.231699,38703,940690


In [69]:
len(grouped_stats)

51158

In [70]:
total_per_hs = grouped_stats.groupby("HS")["v_sum"].transform("sum")
grouped_stats["v_share"] = grouped_stats["v_sum"] / total_per_hs
grouped_stats = grouped_stats[grouped_stats["v_share"] >= 0.01].drop(columns="v_share")

In [71]:
len(grouped_stats)

6506

Rows we lost, rows we didn't have in the CPC-HS mapping

# 4. Merge with country codes

In [72]:
merged = grouped_stats.merge(
    country_codes[['country_code', 'country_name', 'country_iso2']],
    left_on='i',
    right_on='country_code',
    how='left'
).rename(columns={'country_name': 'exporter',
                 'country_iso2': 'exporter_iso2'
                 })

In [73]:
merged

,i,k,v_sum,q_sum,mean_unit,min_unit,max_unit,unit_value,CPC,HS,country_code,exporter,exporter_iso2
0,4,100410,883.618,1893.000,0.466782,0.466782,0.466782,0.466782,1171,100410,4,Afghanistan,AF
1,8,121190,48789.133,10545.222,8.499452,0.932443,45.045455,4.626658,1930,121190,8,Albania,AL
2,8,630520,15982.727,1427.309,26.025642,9.306683,87.000000,11.197804,36410,630520,8,Albania,AL
3,8,720241,149757.144,55194.545,2.868185,1.477375,4.002427,2.713260,41113,720241,8,Albania,AL
4,12,252310,346404.892,5337556.748,0.775937,0.012033,9.611111,0.064900,37430,252310,12,Algeria,DZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6501,894,252220,16854.201,109045.922,0.159525,0.143588,0.172446,0.154561,37420,252220,894,Zambia,ZM
6502,894,252310,42135.148,388693.152,0.121162,0.099819,0.135293,0.108402,37430,252310,894,Zambia,ZM
6503,894,260400,196812.133,100042.918,9.196696,1.247038,43.764706,1.967277,14220,260400,894,Zambia,ZM
6504,894,720211,14495.553,16658.444,1.029604,0.670530,1.592000,0.870162,41112,720211,894,Zambia,ZM


# 5. csv output

In [74]:
final_df = merged[['CPC', 'HS', 'i', 'exporter_iso2', 'exporter', 'unit_value', 'mean_unit', 'min_unit', 'max_unit', 'v_sum']].copy()
final_df

,CPC,HS,i,exporter_iso2,exporter,unit_value,mean_unit,min_unit,max_unit,v_sum
0,1171,100410,4,AF,Afghanistan,0.466782,0.466782,0.466782,0.466782,883.618
1,1930,121190,8,AL,Albania,4.626658,8.499452,0.932443,45.045455,48789.133
2,36410,630520,8,AL,Albania,11.197804,26.025642,9.306683,87.000000,15982.727
3,41113,720241,8,AL,Albania,2.713260,2.868185,1.477375,4.002427,149757.144
4,37430,252310,12,DZ,Algeria,0.064900,0.775937,0.012033,9.611111,346404.892
...,...,...,...,...,...,...,...,...,...,...
6501,37420,252220,894,ZM,Zambia,0.154561,0.159525,0.143588,0.172446,16854.201
6502,37430,252310,894,ZM,Zambia,0.108402,0.121162,0.099819,0.135293,42135.148
6503,14220,260400,894,ZM,Zambia,1.967277,9.196696,1.247038,43.764706,196812.133
6504,41112,720211,894,ZM,Zambia,0.870162,1.029604,0.670530,1.592000,14495.553


In [75]:
output_fp = 'ecoinvent_cpc_unit_values.csv'
final_df.to_csv(output_fp, index=False)

# 6. Check units and add conversion factors

In [76]:
import bw2data as bd

In [77]:
bd.projects.set_current('bw25_ei310')

In [78]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere
	ecoinvent-3.10.1-cutoff

In [79]:
db = bd.Database('ecoinvent-3.10-cutoff')

In [80]:
cpc_codes = set(final_df['CPC'].astype(int))
len(cpc_codes)

467

In [81]:
unit_info = {}
for act in db:
    if 'classifications' in act and act['classifications']:
        for cls_type, cls_value in act['classifications']:
            if cls_type == 'CPC':
                try:
                    code = int(cls_value.split(':')[0].strip())
                    if code in cpc_codes:
                        unit = act.get('unit', '').lower()
                        ref_product = act.get('reference product', '').lower()
                        entry = {
                            'name': act['name'],
                            'reference product': ref_product,
                            'unit': unit,
                            'location': act.get('location', ''),
                            'key': act.key
                        }
                        if code not in unit_info:
                            unit_info[code] = []
                        unit_info[code].append(entry)
                except ValueError:
                    continue

In [82]:
unit_info

{}

# 7. Handling non-kg units for the JSON

In [83]:
conversion_rules = {
    'ngas_highpressure': {
        'unit': 'cubic meter',
        'keyword': 'natural gas, high pressure',
        'factor': 0.735 # kg/m3 | Density stated in EI description
    }, 
    'ngas_liquified': {
        'unit': 'cubic meter',
        'keyword': 'natural gas, liquified',
        'factor': 0.735 # kg/m3 | EI: The reference flow refers to 1 standard cubic meter of natural gas in the gaseous state
    }, 
    'ngas_liquified': {
        'unit': 'cubic meter',
        'keyword': 'natural gas, liquefied',
        'factor': 0.735 # kg/m3 | EI: The reference flow refers to 1 standard cubic meter of natural gas in the gaseous state
    },
    'ngas_lowpressure': {
        'unit': 'cubic meter',
        'keyword': 'natural gas, low pressure',
        'factor': 0.735 # kg/m3 | EI: a density of 0.735kg/m3 for natural gas are applied, in line with the global statistics (referring to standard cubic meters, Sm3, measured at 15°C and 1013 mbar)
    }, 
    'ngas_vented': {
        'unit': 'cubic meter',
        'keyword': 'natural gas, vented',
        'factor': 0.735
    }, 
    'ngas_sweetening': {
        'unit': 'cubic meter',
        'keyword': 'sweetening, natural gas',
        'factor': 0.735
    },
    'biogas': {
        'unit': 'cubic meter',
        'keyword': 'biogas',
        'factor': 0.735
    },
    'electrolyte_nickel': {
        'unit': 'cubic meter',
        'keyword': 'electrolyte, nickel-rich',
        'factor': 1300 # Estimate - More dense than pure water
    },
    'hardwood': {
        'unit': 'cubic meter',
        'keyword': 'hardwood',
        'factor': 800 # Estimate
    },
    'softwood': {
        'unit': 'cubic meter',
        'keyword': 'softwood',
        'factor': 500 # Estimate 
    },
    'timber': {
        'unit': 'cubic meter',
        'keyword': 'timber',
        'factor': 650 # Estimate
    },
    'building_machine': {
        'unit': 'unit',
        'keyword': 'building machine',
        'factor': 7000 # 7000kg, 100% steel
    },
    'cookstove': {
        'unit': 'unit',
        'keyword': 'cookstove',
        'factor': 57.9 # EI: The appliance is 57.9 kg in mass 
    },
    'ng_biomethane': {
        'unit': 'cubic meter',
        'keyword': 'mixed natural gas and biomethane, high pressure',
        'factor': 0.752 # Biomethane density
    },
    'concrete': {
        'unit': 'cubic meter',
        'keyword': 'concrete',
        'factor': 2300 # Estimate 
    },
    'cement': {
        'unit': 'cubic meter',
        'keyword': 'cement',
        'factor': 1500 # Estimate 
    },
    'marine_engine': {
        'unit': 'unit',
        'keyword': 'marine engine',
        'factor':  1000 # EI: per 1000 kg of engine, based on an average marine engine composition.
    },
    'mattress': {
        'unit': 'unit',
        'keyword': 'mattress',
        'factor': 25 #  EI: dimension of 1 m width and 2 m length. Estimate: 1x2x0.25m3 x 50 kg/m3 = 25kg/unit
    },
    'cage': {
        'unit': 'meter',
        'keyword': 'cage',
        'factor': 150 # Estimate from materials in EI
    },
    'greenhouse_glass': {
        'unit': 'square meter-year',
        'keyword': 'greenhouse, glass walls',
        'factor': 1.2 # Estimate from materials in EI
    },
    'condensate_lightoil': {
        'unit': 'cubic meter',
        'keyword': 'condensate from light oil boiler',
        'factor': 1000 # Estimate: watertreatment
    },
    'ultraviolet_lamp': {
        'unit': 'unit',
        'keyword': 'ultraviolet lamp',
        'factor': 0.3824 # Estimate from materials in EI
    },
    'compact_fluorescent_lamp': {
        'unit': 'unit',
        'keyword': 'compact fluorescent lamp',
        'factor': 0.075 # EI: average compact fluorescent lamp which is circa 75 g in mass
    },
    'locomotive': {
        'unit': 'unit',
        'keyword': 'locomotive',
        'factor': 84000 # EI: It has a lifespan of 40 years and a total weight of 84 tons
    },
    'hard_coal': {
        'unit': 'megajoule',
        'keyword': 'hard coal',
        'factor': 0.03184713375 # EI: Low heating value is 31.4 MJ/kg
    },
    'lignite': {
        'unit': 'megajoule',
        'keyword': 'lignite',
        'factor': 0.05128205128 # EI: Low heating value is 19.5 MJ/kg
    },
    'peat_moss': {
        'unit': 'cubic meter',
        'keyword': 'peat moss',
        'factor': 100 # EI: A density of 100 kg/m³ is considered for dry peat moss in loose form
    },
    'strand_board': {
        'unit': 'cubic meter',
        'keyword': 'oriented strand board',
        'factor': 607 # EI
    },
    'energy_feed': {
        'unit': 'megajoule',
        'keyword': 'energy feed, gross',
        'factor': 0.05 # I take a gross average
    },
    'soybean_beverage': {
        'unit': 'litre',
        'keyword': 'soybean beverage',
        'factor': 1 # Estimate
    },
    'coal_gas_mj': {
        'unit': 'megajoule',
        'keyword': 'coal gas',
        'factor': 0.03496503496 # EI: Hard coal coke is assumed to have a low heating value 28.6 MJ/kg and bulk density is 530 kg/m3
    },
    'biomethane_mj': {
        'unit': 'megajoule',
        'keyword': 'biomethane',
        'factor': 0.02150537634 # EI: biomethane, low pressure' is a non-fossil fuel with a calorific value of 46.50 MJ/kg
    },
    'biogas_mj': {
        'unit': 'megajoule',
        'keyword': 'biogas',
        'factor': 0.05059392872 # EI: biogas' is a non-fossil fuel with a net calorific value of 22.73 MJ/m3. Density assumed: 1.15 kg/m3 -> 19.77 MJ/kg 
    },
    'synthetic_gas_mj': {
        'unit': 'megajoule',
        'keyword': 'synthetic gas', # EI: synthetic gas' is a non-fossil fuel with a calorific value of 6.21 MJ/m3. Density is 1.15 kg/Nm3 -> 5.4 MJ/kg
        'factor': 0.18518518518 # 
    },
    'md_fibreboard': {
        'unit': 'cubic meter',
        'keyword': 'medium density fibreboard',
        'factor': 684 # EI: 684 kg/m3
    },
    'fibreboard_soft_latex': {
        'unit': 'cubic meter',
        'keyword': 'fibreboard, soft, latex bonded',
        'factor': 240 # EI: 240 kg/m3
    },
    'fibreboard_soft': {
        'unit': 'cubic meter',
        'keyword': 'fibreboard, soft',
        'factor': 140 # EI: 140 kg/m3
    },
    'fibreboard_hard': {
        'unit': 'cubic meter',
        'keyword': 'fibreboard, hard',
        'factor': 956 # EI: 956 kg/m3
    },
    'plywood ': {
        'unit': 'cubic meter',
        'keyword': 'plywood',
        'factor': 600 # Estimate: 600kg/m3
    },
    'eur_pallet': {
        'unit': 'unit',
        'keyword': 'eur-flat pallet',
        'factor': 22 #  It represents an average pallet with a mass of 22 kg that it is mainly made out of wood.
    },
    'beverage_carton': {
        'unit': 'square meter',
        'keyword': 'beverage carton',
        'factor': 0.25 # Estimate: 250g/m2
    },
    'coke_mj': {
        'unit': 'megajoule',
        'keyword': 'coke',
        'factor': 0.03496503496 # EI: coke' is a fossil fuel with a calorific value of 28.6 MJ/kg
    },
    'window_frame_1.5': {
        'unit': 'square meter',
        'keyword': 'window frame, wood, u=1.5 w/m2k',
        'factor': 80.2 # EI: 1 m2 of visible wooden window frame weighs 80.2 kg
    },
    'door_inner_glasswood': {
        'unit': 'square meter',
        'keyword': 'door, inner, glass-wood',
        'factor': 27.6 # EI: 1 m2 of the wooden inner door weighs 27.6 kg
    },
    'joist_wood': {
        'unit': 'meter',
        'keyword': 'joist, engineered wood',
        'factor': 10 # Estimate: 10kg
    },
    'door_outer_woodglass': {
        'unit': 'square meter',
        'keyword': 'door, outer, wood-glass',
        'factor': 36.5 # EI:  m2 of the aluminium planked massive wood-glass outer door weighs 36.5 kg
    },
    'door_inner_wood': {
        'unit': '',
        'keyword': 'door, inner, wood',
        'factor': 27.6 # EI: 1 m2 of the wooden inner door weighs 27.6 kg
    },
    'diesel_fishingvessel_mj': {
        'unit': 'megajoule',
        'keyword': 'diesel, burned in fishing vessel',
        'factor': 0.02336448598 # EI: diesel' is a fossil fuel with a calorific value of 42.8 MJ/kg
    },
    'uranium_enriched': {
        'unit': 'unit',
        'keyword': 'uranium, enriched',
        'factor': 0.2 # Estimate
    },
    'spent_nuclear_fuel': {
        'unit': 'cubic meter',
        'keyword': 'conditioned spent nuclear fuel',
        'factor': 4000 # Estimate
    },
    'compressed_air_800': {
        'unit': 'cubic meter',
        'keyword': 'compressed air, 800 kpa gauge',
        'factor': 9.5 # Calculated with rho = P/RT -> 9.5 kg/m3
    },
    'compressed_air_1000': {
        'unit': 'cubic meter',
        'keyword': 'compressed air, 1000 kpa gauge',
        'factor': 11.89 # Calculated with rho = P/RT -> 11.89 kg/m3
    },
    'compressed_air_1200': {
        'unit': 'cubic meter',
        'keyword': 'compressed air, 1200 kpa gauge',
        'factor': 14.26 # Calculated with rho = P/RT -> 14.26 kg/m3
    },
    'compressed_air_700': {
        'unit': 'cubic meter',
        'keyword': 'compressed air, 700 kpa gauge',
        'factor': 8.32 # Calculated with rho = P/RT -> 8.32 kg/m3
    },
    'compressed_air_600': {
        'unit': 'cubic meter',
        'keyword': 'compressed air, 600 kpa gauge',
        'factor': 7.13 # Calculated with rho = P/RT -> 7.13 kg/m3
    },
    'meranti': {
        'unit': 'cubic meter',
        'keyword': 'meranti',
        'factor': 675 # Estimate: https://www.wood-database.com/dark-red-meranti/
    },    
    'paranapine': {
        'unit': 'cubic meter',
        'keyword': 'paraná pine',
        'factor': 545 # Estimate: https://www.wood-database.com/parana-pine/
    },  
    'azobe': {
        'unit': 'cubic meter',
        'keyword': 'azobe',
        'factor': 1065 # Estimate: https://www.wood-database.com/ekki/
    },  
    'eucalyptus': {
        'unit': 'cubic meter',
        'keyword': 'eucalyptus',
        'factor': 1130 # https://www.wood-database.com/?s=eucalyptus
    },  
    'synthetic_gas': {
        'unit': 'cubic meter',
        'keyword': 'synthetic gas',
        'factor': 1.15 # EI: Density is 1.15 kg/Nm3
    },  
    'biomethane': {
        'unit': 'cubic meter',
        'keyword': 'biomethane',
        'factor': 0.752 # EI: Density is 0.752 kg/Nm3
    },  
    'bf_gas_mj': {
        'unit': 'megajoule',
        'keyword': 'blast furnace gas',
        'factor': 0.5 # Estimate: 2 MJ/kg
    },  
    'particleboard': {
        'unit': 'cubic meter',
        'keyword': 'particleboard',
        'factor': 680 # EI: 680kg/m³
    },  
    'cladding_softwood': {
        'unit': 'square meter',
        'keyword': 'wood cladding, softwood',
        'factor': 6.9 # EI: For every 1 m2, there is an equivalent of 6.9 kg of wood cladding
    },  
    'door_inner_wood': {
        'unit': 'square meter',
        'keyword': 'door, inner, wood',
        'factor': 27.6 # EI: 1 m2 of the wooden inner door weighs 27.6 kg
    },  
    'low_radioactive_waste': {
        'unit': 'cubic meter',
        'keyword': 'low level radioactive waste',
        'factor': 500 # ESTIMATe
    },  
    'high_radioactive_waste': {
        'unit': 'cubic meter',
        'keyword': 'high level radioactive waste',
        'factor': 4000 # ESTIMATe
    },  
    'green_manure': {
        'unit': 'hectare',
        'keyword': 'green manure',
        'factor': 2300 # EI: The dry matter yield is 2300 kg/ha. 
    },  
    'tailing_uranium': {
        'unit': 'cubic meter',
        'keyword': 'tailing, from uranium milling',
        'factor': 1600 #  ESTIMATE
    },  
    'gluing_mill': {
        'unit': 'unit',
        'keyword': 'gluing mill',
        'factor': 0 # # BACI does not account for infrastructure  
    },  
    'ion-exchanger': {
        'unit': 'unit',
        'keyword': 'ion-exchanger for water treatment',
        'factor': 9900 # Estimate from the materials in EI
    },  
    'solar_glass': {
        'unit': 'square meter',
        'keyword': 'anti-reflex-coating, etching, solar glass',
        'factor': 0.15 # Estimate from the materials in EI
    },  
    'hydraulic_fracturing_fluid': {
        'unit': 'cubic meter',
        'keyword': 'hydraulic fracturing fluid',
        'factor': 1000 # proxy: water
    },  
    'glazing_0.5': {
        'unit': 'square meter',
        'keyword': 'glazing, triple, u<0.5 w/m2k',
        'factor': 30 # EI: 1 m2 visible glazing area has a final weight of 30 kg. 
    },  
    'glazing_11': {
        'unit': 'square meter',
        'keyword': 'glazing, double, u<1.1 w/m2k',
        'factor': 20 # EI: 1 m2 visible glazing area has a final weight of 20 kg.
    },  
    'horticultural_fleece': {
        'unit': 'square meter',
        'keyword': 'horticultural fleece',
        'factor': 0.017 #  EI: he fleece density is 17g/m2
    },  
    'window_16': {
        'unit': 'square meter',
        'keyword': 'window frame, poly vinyl chloride, u=1.6 w/m2k',
        'factor': 94.5 # EI: 1 m2 of visible plastic window frame weighs 94.5 kg. 
    },  
    'container_plastic': {
        'unit': 'unit',
        'keyword': 'container, for collection of post-consumer waste plastic for recycling',
        'factor': 0.02 # Estimate from the materials in EI
    },  
    'used_window_plastic': {
        'unit': 'square meter',
        'keyword': 'used window frame, plastic',
        'factor': 86.53 #  Estimate from the materials in EI
    },  
    'sealing_tape': {
        'unit': 'meter',
        'keyword': 'sealing tape, aluminium/pe, 50 mm wide',
        'factor': 0.057 #  EI: Weight 0.057 kg/m
    },  
    'layered_board': {
        'unit': 'cubic meter',
        'keyword': 'three and five layered board',
        'factor': 600 # Same ESTIMATE as plywood
    },  
    'bev_carton_1L': {
        'unit': 'unit',
        'keyword': 'beverage carton, 1 l',
        'factor': 0.03435 #  EI: The beverage carton's volume is 1 L and its mass is 34.35 g
    },  
    'pe_pipe_75': {
        'unit': 'meter',
        'keyword': 'polyethylene pipe, corrugated, dn 75',
        'factor': 0.33 #  EI: Extruded polyethylene tube with a weight of 0.33 kg/m
    },  
    'pe_pipe_200': {
        'unit': 'meter',
        'keyword': 'polyethylene pipe, dn 200, sdr 41',
        'factor': 3 # EI: Weight 3 kg/m
    },  
    'insulation_400': {
        'unit': 'meter',
        'keyword': 'insulation spiral-seam duct, rockwool, dn 400, 30 mm',
        'factor': 3.3 # EI: Weight 3.3 kg/m
    },  
    'stone_wool_factory': {
        'unit': 'unit',
        'keyword': 'stone wool factory',
        'factor': 0 # BACI does not account for infrastructure  
    },  
    'residual_wood': {
        'unit': 'cubic meter',
        'keyword': 'residual wood, dry',
        'factor': 500 # ESTIMATE  
    }, 
    'dust_multicyclone': {
        'unit': 'unit',
        'keyword': 'dust collector, multicyclone',
        'factor': 1500 #  EI bill of materials: 1500kg steel 
    }, 
    'dust_collector_industrial': {
        'unit': 'unit',
        'keyword': 'dust collector, electrostatic precipitator, for industrial use',
        'factor': 1682 #  EI bill of materials: 1500kg steel + 182kg   
    }, 
    'dust_collector_domestic': {
        'unit': 'unit',
        'keyword': 'dust collector, electrostatic precipitator, for domestic use',
        'factor': 6 # Estimate from EI bill of materials
    }, 
    'gh_walls_roof': {
        'unit': 'square meter-year',
        'keyword': 'greenhouse, plastic walls and roof',
        'factor': 1.5 # Estimate from bill of materials  
    }, 
    'used_window_wood': {
        'unit': 'square meter',
        'keyword': 'used window frame, wood',
        'factor': -77.4 # Estimated from materials in EI   
    }, 
    'used_silencer_315': {
        'unit': 'unit',
        'keyword': 'used silencer steel, dn 315',
        'factor': 20 # EI: Weight of element 20 kg  
    }, 
    'used_air_roof_400': {
        'unit': 'unit',
        'keyword': 'used exhaust air roof hood steel, dn 400',
        'factor': 17 # EI: Weight per element 17 kg
    }, 
    'used_room_overflow': {
        'unit': 'unit',
        'keyword': 'used room-connecting overflow element steel, approx. 40 m3/h',
        'factor': 1.1 #   Weight per element 1.1 kg
    }, 
    'used_silencer_125': {
        'unit': 'unit',
        'keyword': 'used silencer steel, dn 125',
        'factor': 8.3 # EI: Weight of element 8.3 kg
    }, 
    'used_air_intake_370': {
        'unit': 'unit',
        'keyword': 'used outside air intake stainless steel, dn 370',
        'factor': 32 #  Weight per element 32 kg
    }, 
    'used_air_plasticsteel': {
        'unit': 'unit',
        'keyword': 'used exhaust air valve in-wall housing, plastic/steel',
        'factor': 0.5 # Weight per element 0.5 kg 
    }, 
    'used_air_120m': {
        'unit': 'unit',
        'keyword': 'used air distribution terminal panel steel, 120 m3/h',
        'factor': 9.5 # Weight per element 9.5 kg
    }, 
    'used_duct_125': {
        'unit': 'meter',
        'keyword': 'used flexible duct aluminium/pet, dn of 125',
        'factor': 0.205 # Weight 0.205 kg/m
    }, 
    'used_tape_50': {
        'unit': 'meter',
        'keyword': 'used sealing tape aluminium/pe, 50 mm wide',
        'factor': 0.057 # Weight 0.057 kg/m
    }, 
    'zinc_coat_pieces': {
        'unit': 'square meter',
        'keyword': 'zinc coat, pieces',
        'factor': 16.67 # EI: Data given per tonne of coated product had to be transformed dividing it by the mean surface area of 60 m2/t
    }, 
    'zinc_coat_coils': {
        'unit': 'square meter',
        'keyword': 'zinc coat, coils',
        'factor': 15.625 #  Data given per tonne of coated product are transformed by dividing them by the mean surface area of 64 m2/t 
    }, 
    'vessel_25': {
        'unit': 'unit',
        'keyword': 'expansion vessel, 25l',
        'factor': 6 # Estimated from EI materials  
    }, 
    'vessel_80': {
        'unit': 'unit',
        'keyword': 'expansion vessel, 80l',
        'factor': 15.67 # Estimated from EI materials  
    }, 
    'sewage_sludge': {
        'unit': 'cubic meter',
        'keyword': 'sewage sludge',
        'factor': 1000 # Estimate: water   
    }, 
    'sawing_slurry': {
        'unit': 'litre',
        'keyword': ' spent sawing slurry from si-wafer cutting',
        'factor': 1.75 # EI: The specific weight of input slurry 1.75kg/l (wet mass).
    }, 
    'chromium_coat': {
        'unit': 'square meter',
        'keyword': 'hard chromium coat, electroplating, steel substrate, 0.14 mm thickness',
        'factor': 2.7 #   Estimate from EI material inputs
    }, 
    'leachate': {
        'unit': 'cubic meter',
        'keyword': 'leachate',
        'factor': 1000 # estimate  
    }, 
    'wastewater': {
        'unit': 'cubic meter',
        'keyword': 'wastewater',
        'factor': 1000 #   
    }, 
    'coat_steel': {
        'unit': 'square meter',
        'keyword': 'selective coat, stainless steel sheet, black chrome',
        'factor': 8.715 # Estimate from EI material inputs   
    }, 
    'powder_coat_steel': {
        'unit': 'square meter',
        'keyword': 'powder coat, steel',
        'factor': 0.45 # Estimate from EI material inputs 
    },
    'tin_sheet_2mm': {
        'unit': 'square meter',
        'keyword': 'tin plated chromium steel sheet, 2 mm',
        'factor': 15.6 # EI material inputs  
    }, 
    'spiral_steel_125': {
        'unit': 'meter',
        'keyword': 'spiral-seam duct, steel, dn 125',
        'factor': 1.9 # EI: Weight 1.9 kg/m
    }, 
    'spiral_steel_400': {
        'unit': 'meter',
        'keyword': 'spiral-seam duct, steel, dn 400',
        'factor': 6 # Weight 6 kg/m
    }, 
    'powder_coat_aluminium': {
        'unit': 'square meter',
        'keyword': 'powder coat, aluminium sheet',
        'factor': 0.18 # EI material inputs  
    }, 
    'aluminium_wire': {
        'unit': 'meter',
        'keyword': 'aluminium around steel bi-metal stranded cable, 3x3.67mm external diameter wire',
        'factor': 0.07 # EI material inputs   
    }, 
    'window_frame_woodmetal_16': {
        'unit': 'square meter',
        'keyword': 'window frame, wood-metal, u=1.6 w/m2k',
        'factor': 83.4 # EI: 1 m2 of visible wooden window frame weighs 83.4 kg.
    }, 
    'door_outer_woodaluminium': {
        'unit': 'square meter',
        'keyword': 'door, outer, wood-aluminium',
        'factor': 38.8 # EI: 1 m2 of the aluminium planked massive wood outer door weighs 38.8 kg
    }, 
    'window_aluminium_16': {
        'unit': 'square meter',
        'keyword': 'window frame, aluminium, u=1.6 w/m2k',
        'factor': 50.7 # EI: 1 m2 of visible aluminium window frame weighs 50.7 kg.
    }, 
    'cladding_aluminium': {
        'unit': 'square meter',
        'keyword': 'cladding, crossbar-pole, aluminium',
        'factor': 86.7 # EI: The aluminium cladding weighs 86.7 kg per m2.
    }, 
    'storage_tank_organics': {
        'unit': 'unit',
        'keyword': 'liquid storage tank, chemicals, organics',
        'factor': 0 # BACI does not include infrastructure
    },
    'slurry_si_wafer': {
        'unit': 'litre',
        'keyword': 'spent sawing slurry from si-wafer cutting',
        'factor': 1.75 # EI: The specific weight of input slurry 1.75kg/l
    }, 
    'used_glazing_11': {
        'unit': 'square meter',
        'keyword': 'used double glazing, u<1.1w/m2k',
        'factor': 26 # 1 m2 visible glazing area has a final weight of 26 kg.
    },
    'decommissioned_road': {
        'unit': 'meter-year',
        'keyword': 'decommissioned road',
        'factor': 2600 # Kg derived frmo transport ton-km: transport over a distance of 20 kilometres of 80% of the excavation to recycling plants
    }, 
    'uranium_tailin_nonradioactive': {
        'unit': 'cubic meter',
        'keyword': 'uranium tailing, non-radioactive emission',
        'factor': 0 # These are emissions   
    }, 
    'used_air_central_600': {
        'unit': 'unit',
        'keyword': 'used air filter central unit, 600 m3/h',
        'factor': 0.4 #   EI: Total weight per element 0.4 kg
    }, 
    'used_refrigeration_co2': {
        'unit': 'unit',
        'keyword': 'used refrigeration machine, carbon dioxide, liquid as refrigerant',
        'factor': 322 # EI material inputs 
    }, 
    'used_train_longdistance': {
        'unit': 'unit',
        'keyword': 'used train, passenger, long-distance',
        'factor': 88355 # EI material inputs 
    }, 
    'used_lorry_28': {
        'unit': 'unit',
        'keyword': 'used lorry, 28 metric ton',
        'factor': 573.4 # EI material inputs   
    }, 
    'used_shipping_45': {
        'unit': 'unit',
        'keyword': 'used intermodal shipping container, 45-foot, high-cube',
        'factor': -4767.895 # EI material inputs  
    }, 
    'used_bicycle': {
        'unit': 'unit',
        'keyword': 'used bicycle',
        'factor': 17 # EI: The data set reflects the disposal of a bike of 17kg
    }, 
    'used_scooter': {
        'unit': 'unit',
        'keyword': 'used motor scooter',
        'factor': 32 #  # EI material inputs  
    }, 
    'used_railway_track': {
        'unit': 'meter-year',
        'keyword': 'used railway track',
        'factor': 38.79 # # EI material inputs    
    }, 
    'used_ebicycle': {
        'unit': 'unit',
        'keyword': 'used electric bicycle',
        'factor': 24 # EI: The data set reflects the disposal of an electric bike of 24kg.
    },
    'used_blower_1200': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit kwlc 1200',
        'factor': 170 # EI: Total weight of component 170 kg
    },
    'waste_ref_gas_mj': {
        'unit': 'megajoule',
        'keyword': 'waste refinery gas',
        'factor': 0 # These are emissions 
    },
    'decommisioned_tram_track': {
        'unit': 'meter-year',
        'keyword': 'decommissioned tram track',
        'factor': 1581.78 # Derived from EI transport ton-km: Consequently is merely accounted transport of the materials over a distance of 20 km. 
    },
    'used_tram': {
        'unit': 'unit',
        'keyword': 'used tram',
        'factor': 1761 # Derived from EI material inputs
    },
    'used_blower_250': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit ge 250 rh',
        'factor': 65 # EI: Total weight of component 65 kg
    },
    'used_spiral_400': {
        'unit': 'meter',
        'keyword': 'used insulation spiral-seam duct rockwool, dn 400',
        'factor': 3.3 # EI: Weight 3.3 kg/m
    },
    'used_blower_600_1200': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit central, 600-1200 m3/h',
        'factor': 143 # EI: Total weight of component 143 kg 
    },
    'used_lorry_16': {
        'unit': 'unit',
        'keyword': 'used lorry, 16 metric ton',
        'factor': 360.2 # Derived from EI material inputs
    },
    'used_train_regional': {
        'unit': 'unit',
        'keyword': 'used train, passenger, regional',
        'factor': 26435.570 # Derived from EI material inputs
    },
    'rainwater_oilstorage': {
        'unit': 'cubic meter',
        'keyword': 'rainwater mineral oil storage',
        'factor': 1000 # assumption: water 
    },
    'used_shipping_40': {
        'unit': 'unit',
        'keyword': 'used intermodal shipping container, 40-foot',
        'factor': 3800 # Derived from EI material inputs
    },
    'used_filter_180_250': {
        'unit': 'unit',
        'keyword': 'used air filter decentralized unit, 180-250 m3/h',
        'factor': 0.22 # Derived from EI material inputs 
    },
    'used_exhaust_valve': {
        'unit': 'unit',
        'keyword': 'used air filter in exhaust air valve',
        'factor': 0.04 # EI: Total weight per element 0.04 kg
    },
    'waste_ng_sour': {
        'unit': 'megajoule',
        'keyword': 'waste natural gas, sour',
        'factor': 0 # These are emissions
    },
    'used_blower_90': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit storkair g 90',
        'factor': 32.7 # EI: Total weight of component 32.7 kg
    },
    'used_domestic_refrigerator': {
        'unit': 'unit',
        'keyword': 'used domestic refrigerator',
        'factor': 58.104054 # Derived from EI material inputs 
    },
    'used_lorry_refrig_16': {
        'unit': 'unit',
        'keyword': 'used lorry with refrigeration machine, 16 metric ton',
        'factor': 6658.856553 # Derived from EI material inputs 
 # 
    },
    'used_shipping_20foot': {
        'unit': 'unit',
        'keyword': 'used intermodal shipping container, 20-foot',
        'factor': 3166.441226 # Derived from EI material inputs 
    },
    'used_train_highspeed': {
        'unit': 'unit',
        'keyword': 'used train, passenger, high-speed',
        'factor': 192600 # Derived from EI material inputs 
    },
    'used_blower_e97': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit avent e 97',
        'factor': 35 # EI: Total weight of component 35 kg
    },
    'used_lorry_40': {
        'unit': 'unit',
        'keyword': 'used lorry, 40 metric ton',
        'factor': 895 # Derived from EI material inputs 
    },
    'used_container_40_highcube': {
        'unit': 'unit',
        'keyword': ' used intermodal shipping container, 40-foot, high-cube',
        'factor': 3811.498596 # Derived from EI material inputs 
    },
    'used_blower_180_250': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit decentralized, 180-250 m3/h',
        'factor': 43.1 # Total weight of component 43.1 kg
    },
    'used_lightcomm_vehicle': {
        'unit': 'unit',
        'keyword': 'used light commercial vehicle',
        'factor': 98.4 # Derived from EI material inputs 
    },
    'used_filter_250': {
        'unit': 'unit',
        'keyword': 'used air filter decentralized unit, 250 m3/h',
        'factor': 0.05 # EI: Total weight per element 0.05 kg
    },
    'used_reefer_40highcube': {
        'unit': 'unit',
        'keyword': 'used reefer, intermodal shipping container, 40-foot, high-cube',
        'factor': -158.233405 # Derived from EI material inputs 
    },
    'waste_ng_sweet_mj': {
        'unit': 'megajoule',
        'keyword': 'waste natural gas, sweet',
        'factor': 0 # These are emissions
    },
    'decommissioned_airport_contaminatedgravel': {
        'unit': 'unit',
        'keyword': 'decommissioned airport infrastructure, contaminated gravel',
        'factor': 0 # These are emissions/landfill
    },
    'heat_carrier_c3h8o2': {
        'unit': 'cubic meter',
        'keyword': 'heat carrier liquid, 40% c3h8o2',
        'factor': 1000 # estimate:water
    },
    'used_blower_kwl250': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit kwl 250',
        'factor': 40 # EI: Total weight of component 40 kg
    },
    'used_dieselelectric_set_18_5kw': {
        'unit': 'unit',
        'keyword': 'used diesel-electric generating set, 18.5kw',
        'factor': 817 # Derived from EI material inputs 
    },
    'used_glazing05': {
        'unit': 'square meter',
        'keyword': 'used triple glazing, u<0.5w/m2k',
        'factor': 30 # From the production dataset: 1 m2 visible glazing area has a final weight of 30 kg.
    },
    'used_ventilationcontrol_decentral': {
        'unit': 'unit',
        'keyword': 'used ventilation control and wiring decentralized unit',
        'factor': 2.18 # EI: Weight of control unit 0.6 kg, weight of wiring 1.58 kg
    },
    'used_ventilationcontrol_central': {
        'unit': 'unit',
        'keyword': 'used ventilation control and wiring central unit',
        'factor': 4.8 # EI: Weight of control unit 0.6 kg, weight of wiring 4.2 kg
    },
    'used_blower_twl700': {
        'unit': 'unit',
        'keyword': 'used blower and heat exchange unit twl-700',
        'factor': 120 # EI: Total weight of component 120 kg
    },
    'used_refrigeration_r134a': {
        'unit': 'unit',
        'keyword': 'used refrigeration machine, r134a as refrigerant',
        'factor': 311.8024348 # Derived from EI material inputs 
    },
    'used_bus': {
        'unit': 'unit',
        'keyword': 'used bus',
        'factor': 705.2 # Derived from EI material inputs 
    },
    'aluminium_wire_367': {
        'unit': 'meter',
        'keyword': 'aluminium around steel bi-metal wire, 3.67mm external diameter',
        'factor': 0.073335418 # Derived from material inputs
    },
    'storage_10000l': {
        'unit': 'unit',
        'keyword': "storage, 10'000 l",
        'factor': 0 # We are excluding built infrastructure
    },
    'oil_storage_3000': {
        'unit': 'unit',
        'keyword': 'oil storage, 3000l',
        'factor': 0 # We are excluding built infrastructure
    },
    'heat_storage_2000': {
        'unit': 'unit',
        'keyword': 'heat storage, 2000l',
        'factor': 0 # We are excluding built infrastructure
    },
    'cogen_unit_64mwth': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, organic rankine cycle, 1mwe, 6.4 mwth',
        'factor': 0 # We are excluding built infrastructure
    },
    'gas_motor_206': {
        'unit': 'unit',
        'keyword': 'gas motor, 206kw',
        'factor': 1400 # EI: It has a 206 kW capacity and weighs 1400 kg
    },
    'gas_motor_chp': {
        'unit': 'unit',
        'keyword': 'gas motor, mini chp plant',
        'factor': 30 # EI: The motor is a water-cooled single-cylinder 4-stroke piston-gas internal combustion engine that weighs 30 kg
    },
    'gas_turbine_10mw': {
        'unit': 'unit',
        'keyword': 'gas turbine, 10mw electrical',
        'factor': 185000 # Derived from EI material inputs
    },
    'gas_turbine_80mw': {
        'unit': 'unit',
        'keyword': 'gas turbine, 80mw, for compressed air energy storage',
        'factor': 167679.2 # Derived from EI material inputs
    },
    'water_pump_22kw': {
        'unit': 'unit',
        'keyword': 'water pump, 22kw',
        'factor': 300 # EI: the total mass of the pump is 300 kg
    },
    'pump_40w': {
        'unit': 'unit',
        'keyword': 'pump, 40w',
        'factor': 2.427 # Derived from EI material inputs 
    },
    'air_compressor_300kw': {
        'unit': 'unit',
        'keyword': 'air compressor, screw-type compressor, 300kw',
        'factor': 4600 # EI: The weight of the compressor is 4600 kg
    },
    'air_compressor_4kw': {
        'unit': 'unit',
        'keyword': 'air compressor, screw-type compressor, 4kw',
        'factor': 140 # EI: The weight of the compressor is 140 kg
    },
    'air_cogen_160': {
        'unit': 'unit',
        'keyword': 'air input/output unit, heat and power co-generation unit, 160kw electrical',
        'factor': 288 # derived from EI material inputs
    },
    'microgasturbine_100kw': {
        'unit': 'unit',
        'keyword': 'micro gas turbine, 100kw electrical',
        'factor': 0 # This is a maintenance process 
    },
    'coal_stove_515': {
        'unit': 'unit',
        'keyword': 'coal stove, 5-15kw',
        'factor': 460.74 # Derived from EI material inputs
    },
    'gas_boiler': {
        'unit': 'unit',
        'keyword': 'gas boiler',
        'factor': 0 # This is a construction process
    },
    'oil_boiler_100kw': {
        'unit': 'unit',
        'keyword': 'oil boiler, 100kw',
        'factor': 588.95 # Derived from EI material inputs 
    },
    'oil_boiler_10kw': {
        'unit': 'unit',
        'keyword': 'oil boiler, 10kw',
        'factor': 149.73 # Derived from EI material inputs 
    },
    'furnace_woodchips_hardwood_50kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, hardwood storage area, 50kw',
        'factor': 644.8 # Derived from EI material inputs | We exclude concrete 
    },
    'furnace_logs_6kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, 6kw',
        'factor': 135.6 # Derived from EI material inputs  
    },
    'furnace_logs_hardwood_30kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, hardwood storage area, 30kw',
        'factor': 506.48 # Derived from EI material inputs | We exclude concrete 
    },
    'furnace_pellets_9kw': {
        'unit': 'unit',
        'keyword': 'furnace, pellets, 9kw',
        'factor': 165.2 # Derived from EI material inputs  
    },
    'furnace_ind_coal_110mw': {
        'unit': 'unit',
        'keyword': 'industrial furnace, coal, 1-10mw',
        'factor': 37260 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_silo_50kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, with silo, 50kw',
        'factor': 1166.5 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_logs_softwood_30kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, softwood storage area, 30kw',
        'factor': 506.48 # Derived from EI material inputs - Concrete excluded
    },
    'blast_furnace': {
        'unit': 'unit',
        'keyword': 'blast furnace',
        'factor': 0 # We are excluding built infrastructure
    },
    'furnace_logs_hardwood_30kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, hardwood storage area, 30kw',
        'factor': 506.48 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_pellets_50kw': {
        'unit': 'unit',
        'keyword': 'furnace, pellet, 50kw',
        'factor': 750.8 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_silo_1000kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, with silo, 1000kw',
        'factor': 52675 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_pellets_25kw': {
        'unit': 'unit',
        'keyword': 'furnace, pellets, 25kw',
        'factor': 575.7 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_silo_300kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, with silo, 300kw',
        'factor': 6196.5 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_hardwood_300kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, hardwood storage area, 300kw',
        'factor': 2409.37 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_average_6kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, average storage area, 6kw',
        'factor': 105.85 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_logs_softwood_6kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, softwood storage area, 6kw',
        'factor': 105.85 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_silo_5000': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, with silo, 5000kw',
        'factor': 120184 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_pellets_15kw': {
        'unit': 'unit',
        'keyword': 'furnace, pellet, 15kw',
        'factor': 614.37 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_average_300kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, average storage area, 300kw',
        'factor': 2409.37 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_logs_hardwood_6kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, hardwood storage area, 6kw',
        'factor': 105.85 # Derived from EI material inputs - Concrete excluded
    },
    'industrial_furnace_1mw_oil': {
        'unit': 'unit',
        'keyword': 'industrial furnace, 1mw, oil',
        'factor': 4765.48 # Derived from EI material inputs
    },
    'furnace_woodchips_average_1000kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, average storage area, 1000kw',
        'factor': 6481.56 # Derived from EI material inputs
    },
    'furnace_logs_softwood_100kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, softwood storage area, 100kw',
        'factor': 1122.55 # Derived from EI material inputs - Concrete excluded
    },
    'furnance_woodchips_average_50kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, average storage area, 50kw',
        'factor': 644.8 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_softwood_50kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, softwood storage area, 50kw',
        'factor': 644.8 # Derived from EI material inputs - Concrete excluded
    },
    'aluminium_melting_furnace': {
        'unit': 'unit',
        'keyword': 'aluminium melting furnace',
        'factor': 0 # This is built infrastructure 
    },
    'furnace_logs_average_30kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, average storage area, 30kw',
        'factor': 506.48 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_logs_30kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, 30kw',
        'factor': 775.3 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_logs_hardwood_100kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, hardwood storage area, 100kw',
        'factor': 1122.55 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_softwood_300kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, softwood storage area, 300kw',
        'factor': 2409.37 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_woodchips_1000kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, softwood storage area, 1000kw',
        'factor': 6481.56 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_hardwood_1000kw': {
        'unit': 'unit',
        'keyword': 'furnace, wood chips, hardwood storage area, 1000kw',
        'factor': 6481.56 # Derived from EI material inputs - Concrete excluded
    },
    'industrial_furnace_ng': {
        'unit': 'unit',
        'keyword': 'industrial furnace, natural gas',
        'factor': 4765.48 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_pellets_silo_300kw': {
        'unit': 'unit',
        'keyword': 'furnace, pellets, with silo, 300kw',
        'factor': 3954 # Derived from EI material inputs - Concrete excluded
    },
    'furnace_logs_average_100kw': {
        'unit': 'unit',
        'keyword': 'furnace, logs, average storage area, 100kw',
        'factor': 1122.55 # Derived from EI material inputs - Concrete excluded
    },
    'aux_heating_elect_5kw': {
        'unit': 'unit',
        'keyword': 'auxiliary heating unit, electric, 5kw',
        'factor': 1.632 # Derived from EI material inputs
    },
    'evacuated_tube_collector': {
        'unit': 'square meter',
        'keyword': 'evacuated tube collector',
        'factor': 27.9473 # Estimated from EI material inputs 
    },
    'blast_oxygen_furnace_converter': {
        'unit': 'unit',
        'keyword': 'blast oxygen furnace converter',
        'factor': 0 # Built infrastructure
    },
    'controlcabinet_cogen_160kw': {
        'unit': 'unit',
        'keyword': 'control cabinet, heat and power co-generation unit, 160kw electrical',
        'factor': 375.9221 # Derived from EI material inputs
    },
    'flatplate_solar_cu': {
        'unit': 'square meter',
        'keyword': 'flat plate solar collector, cu absorber',
        'factor': 27.98328 # Estimated from EI material inputs 
    },
    'conveyor_belt': {
        'unit': 'meter',
        'keyword': 'conveyor belt',
        'factor': 1073.2 # Derived from EI material inputs - Concrete excluded
    },
    'blower_decentralized_180_250': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, decentralized, 180-250 m3/h',
        'factor': 43.1 # EI: Total weight of component 43.1 kg
    },
    'blower_ge250rh': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, ge 250 rh',
        'factor': 65 # Total weight of component 65 kg
    },
    'blower_kwl250': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, kwl 250',
        'factor': 40 # EI: Total weight of component 40 kg
    },
    'blower_central_600_1200': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, central, 600-1200 m3/h',
        'factor': 143 # EI: Total weight of component 143 kg
    },
    'blower_twl-700': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, twl-700',
        'factor': 120 # EI: Total weight of component 120 kg
    },
    'blower_kwlc1200': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, kwlc 1200',
        'factor': 170 # EI: Total weight of component 170 kg
    },
    'blower_avent_e97': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, avent e 97',
        'factor': 35 # EI: Total weight of component 35 kg
    },
    'blower_storkair_g90': {
        'unit': 'unit',
        'keyword': 'blower and heat exchange unit, storkair g 90',
        'factor': 32.7 # EI: Total weight of component 32.7 kg
    },
    'borehole_150m': {
        'unit': 'unit',
        'keyword': 'borehole heat exchanger, 150m',
        'factor': 323 # Estimated from EI material inputs
    },
    '': {
        'unit': 'meter',
        'keyword': 'flexible duct, aluminium/pet, dn of 125',
        'factor': 0.205 # EI: Weight 0.205 kg/m
    },
    'heatpump_brinewater_10kw': {
        'unit': 'unit',
        'keyword': 'heat pump, brine-water, 10kw',
        'factor': 132.79 # Derived from EI material inputs 
    },
    'absorption_chiller_100kw': {
        'unit': 'unit',
        'keyword': 'absorption chiller, 100kw',
        'factor': 8080 # Estimated from EI material inputs
    },
    'heat_pump_minichp': {
        'unit': 'unit',
        'keyword': 'heat pump, for mini chp plant',
        'factor': 50 # Derived from EI material inputs
    },
    'heatpump_diffusion_4kw_future': {
        'unit': 'unit',
        'keyword': 'heat pump, diffusion absorption, 4kw, future',
        'factor': 409.1 # Estimated from EI material inputs
    },
    'refrigeration_co2_as_refrig': {
        'unit': 'unit',
        'keyword': 'refrigeration machine, carbon dioxide, liquid as refrigerant',
        'factor': 380 # EI: It has a lifetime of 10 years and a total mass of 380 kg
    },
    'refrigeration_machine_r134a': {
        'unit': 'unit',
        'keyword': 'refrigeration machine, r134a as refrigerant',
        'factor': 380 # EI: It has a lifetime of 10 years and a total mass of 380 kg
    },
    'heatpump_cogen_160kw': {
        'unit': 'unit',
        'keyword': 'heat pump, heat and power co-generation unit, 160kw electrical',
        'factor': 1180 # Derived from EI material inputs 
    },
    'heatpump_30kw': {
        'unit': 'unit',
        'keyword': 'heat pump, 30kw',
        'factor': 132.79 # [brine-water 10kw] x 3 (This is how ei models it)
    },
    'airfilter_central_600': {
        'unit': 'unit',
        'keyword': 'air filter, central unit, 600 m3/h',
        'factor': 0.4 # EI: Total weight per element 0.4 kg
    },
    'airdistribution_steel_120': {
        'unit': 'unit',
        'keyword': 'air distribution terminal panel, steel, 120 m3/h',
        'factor': 9.5 # Weight per element 9.5 kg
    },
    'airfilter_valve': {
        'unit': 'unit',
        'keyword': 'air filter, in exhaust air valve',
        'factor': 0.04 # EI: Total weight per element 0.04 kg
    },
    'ultrafiltration_module': {
        'unit': 'unit',
        'keyword': 'ultrafiltration module',
        'factor': 19.86875512 # Estimated from EI material inputs 
    },
    'cogen_unit_500kw': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 500kw electrical, components for electricity only',
        'factor': 0 # construction process | no weight stated
    },
    'airfilter_decentralized_180250': {
        'unit': 'unit',
        'keyword': 'air filter, decentralized unit, 180-250 m3/h',
        'factor': 0.22 # EI: Total weight per element 0.22 kg
    },
    'seawater_osmosis_module': {
        'unit': 'square meter',
        'keyword': 'seawater reverse osmosis module',
        'factor': 0.57681484 # estimated from EI material inputs 
    },
    'airfilter_decentralized_250': {
        'unit': 'unit',
        'keyword': 'air filter, decentralized unit, 250 m3/h',
        'factor': 0.05 #  Total weight per element 0.05 kg
    },
    'exhaustairvalve_dn125': {
        'unit': 'unit',
        'keyword': 'exhaust air valve, in-wall housing, plastic/steel, dn 125',
        'factor': 0.5 # EI: Weight per element 0.5 kg
    },
    'exhaustair_roof_steel_dn400': {
        'unit': 'unit',
        'keyword': 'exhaust air roof hood, steel, dn 400',
        'factor': 17 # EI: Weight per element 17 kg
    },
    'outside_airintake_steel_dn370': {
        'unit': 'unit',
        'keyword': 'outside air intake, stainless steel, dn 370',
        'factor': 32 # EI: Weight per element 32 kg
    },
    'heatdistribution_hydronic_150': {
        'unit': 'unit',
        'keyword': 'heat distribution equipment, hydronic radiant floor heating, 150m2',
        'factor': 1193 # Estimated from EI material inputs | Sand not included 
    },
    'exhaustair_steel_al_85365': {
        'unit': 'unit',
        'keyword': 'exhaust air outlet, steel/aluminium, 85x365 mm',
        'factor': 2.5 # Weight per element 2.5 kg
    },
    'supplyair_steel_dn75': {
        'unit': 'unit',
        'keyword': 'supply air inlet, steel/ss, dn 75',
        'factor': 2.2 # EI: Weight per element 2.2 kg
    },
    'chip_stationary_electric': {
        'unit': 'unit',
        'keyword': 'chipper, stationary, electric',
        'factor': 2500 # EI: The chopper mainly consists of unalloyed steel plates and has a total weight of 2.5 t.
    },
    'forwarder': {
        'unit': 'unit',
        'keyword': 'forwarder',
        'factor': 17025.51628 # Estimated from EI material inputs 
    },
    'chipper_mobile_diesel': {
        'unit': 'unit',
        'keyword': 'chipper, mobile, diesel',
        'factor': 72918.75 # Estimated from EI material inputs 
    },
    'mobile_yarder_truckmounted': {
        'unit': 'unit',
        'keyword': 'mobile cable yarder, truck-mounted, incl. processor',
        'factor': 47250.34 # Estimated from EI material inputs 
    },
    'skidder': {
        'unit': 'unit',
        'keyword': 'skidder',
        'factor': 15510.6 # Estimated from EI material inputs 
    },
    'forestry_harvester': {
        'unit': 'unit',
        'keyword': 'forestry harvester',
        'factor': 23071.111 # Estimated from EI material inputs 
    },
    'energy_wood_harvester': {
        'unit': 'unit',
        'keyword': 'energy wood harvester',
        'factor': 21400 # EI: It has an average engine power of 135 kW and a total mass of 21,400 kg
    },
    'cable_yarder_sledwinch': {
        'unit': 'unit',
        'keyword': 'cable yarder with sled winch',
        'factor': 5140 # EI: it has a total mass of 5,140 kg
    },
    'mobile_yarder_trailermounted': {
        'unit': 'unit',
        'keyword': 'mobile cable yarder, trailer-mounted',
        'factor': 19121 # Estimated from EI material inputs 
    },
    'abrasive_blasting_alumina': {
        'unit': 'square meter',
        'keyword': 'abrasive blasting, alumina, carbon steel substrate',
        'factor': 0 # This is not a product. It is a blasting process 
    },
    'nonfe_smelter': {
        'unit': 'unit',
        'keyword': 'non-ferrous metal smelter',
        'factor': 0 # Excluded: This is built infrastructure
    },
    'rolling_mill': {
        'unit': 'unit',
        'keyword': 'rolling mill',
        'factor': 0 # Excluded: This is built infrastructure
    },
    'hydraulic_digger': {
        'unit': 'unit',
        'keyword': 'hydraulic digger',
        'factor': 15000 # Derived from EI material inputs
    },
    'refrigerator': {
        'unit': 'unit',
        'keyword': 'refrigerator',
        'factor': 60 # EI: This product represents an average refrigerator which is approximately 60 kg in mass
    },
    'dishwasher': {
        'unit': 'unit',
        'keyword': 'dishwasher',
        'factor': 50 # EI: The dishwasher is circa 50 kg in mass
    },
    'dryer': {
        'unit': 'unit',
        'keyword': 'dryer',
        'factor': 50 # EI: The appliance is 50 kg in mass
    },
    'washing_machine': {
        'unit': 'unit',
        'keyword': 'washing machine',
        'factor': 75 # ei: average washing machine which is 75 kg in mass
    },
    'ventilation_dwellings_1x720': {
        'unit': 'square meter-year',
        'keyword': 'ventilation of dwellings, central, 1 x 720 m3/h',
        'factor': 0 # Exclude
    },
    'ventilation_dwellings_6x120': {
        'unit': 'square meter-year',
        'keyword': 'ventilation of dwellings, decentralized, 6 x 120 m3/h',
        'factor': 0 # Exclude 
    },
    'ventilation_system_6x120_polyethylene': {
        'unit': 'unit',
        'keyword': 'ventilation system, decentralized, 6 x 120 m3/h, polyethylene ducts',
        'factor': 0 # Exclude 
    },
    'ventilation_system_6x120_polyethylene_earthtubes': {
        'unit': 'unit',
        'keyword': 'ventilation system, decentralized, 6 x 120 m3/h, polyethylene ducts, with earth tube heat exchanger',
        'factor': 0 # Exclude
    },
    'ventilation_system_6x120_steel_earthtubes': {
        'unit': 'unit',
        'keyword': 'ventilation system, decentralized, 6 x 120 m3/h, steel ducts, with earth tube heat exchanger',
        'factor': 0 # Exclude
    },
    'ventilation_system_6x120_steel': {
        'unit': 'unit',
        'keyword': 'ventilation system, decentralized, 6 x 120 m3/h, steel ducts',
        'factor': 0 # Exclude
    },
    'ventilation_system_1x720_polyethylene_earthtubes': {
        'unit': 'unit',
        'keyword': 'ventilation system, central, 1 x 720 m3/h, polyethylene ducts, with earth tube heat exchanger',
        'factor': 0 # Exclude 
    },
    'coffee_maker': {
        'unit': 'unit',
        'keyword': 'coffee maker',
        'factor': 1.9 # EI: verage coffee maker which is approximately 1.9 kg in mass and has a rating of 640 W
    },
    'microwave_oven': {
        'unit': 'unit',
        'keyword': 'microwave oven',
        'factor': 10.6 # ei: average microwave oven which is approximately 10.6 kg in mass, has a rating of 1150 W and capacity of 17 liters
    },
    'hot_watertank_600l': {
        'unit': 'unit',
        'keyword': 'hot water tank, 600l',
        'factor': 283 # Estimated from EI material inputs | Sand excluded 
    },
    'ventilation_system_1x720_steel_earthtubes': {
        'unit': 'unit',
        'keyword': 'ventilation system, central, 1 x 720 m3/h, steel ducts, with earth tube heat exchanger',
        'factor': 0 # Exclude 
    },
    'ventilationduct_connection_steel_100x50': {
        'unit': 'unit',
        'keyword': 'ventilation duct, connection piece, steel, 100x50 mm',
        'factor': 0.2 # EI: Total weight per element 0.2 kg
    },
    'roomconnecting_overflow_steel_40': {
        'unit': 'unit',
        'keyword': 'room-connecting overflow element, steel, approx. 40 m3/h',
        'factor': 1.1 # Weight per element 1.1 kg
    },
    'ventilationduct_steel_100x50': {
        'unit': 'meter',
        'keyword': 'ventilation duct, steel, 100x50 mm',
        'factor': 1.5 # EI: Weight 1.5 kg/m
    },
    'ventilation_controlwiring_decentralized': {
        'unit': 'unit',
        'keyword': 'ventilation control and wiring, decentralized unit',
        'factor': 2.18 # EI: Weight of control unit 0.6 kg, weight of wiring 1.58 kg.
    },
    'ventilationduct_elbow90_steel_100x50': {
        'unit': 'unit',
        'keyword': 'ventilation duct, elbow 90°, steel, 100x50 mm',
        'factor': 0.27 # EI: Total weight per element 0.27 kg
    },
    'ventilation_controlwiring_centralized': {
        'unit': 'unit',
        'keyword': 'ventilation control and wiring, central unit',
        'factor': 4.8 # EI: Weight of control unit 0.6 kg, weight of wiring 4.2 kg
    },
    '': {
        'unit': 'unit',
        'keyword': 'paper machine',
        'factor': 4500000 # EI: It represent a big offset paper machine, used e.g. for newsprint production, with a total mass of 4,500 tonnes and a lifetime of 50 years.
    },
    'tonner_laserprinter_blackwhite': {
        'unit': 'unit',
        'keyword': 'toner module, laser printer, black/white',
        'factor': 1.14 # EI: It has a mass of 880 gr and an additional mass of 260 gr for the toner
    },
    'tonner_laserprinter_colour': {
        'unit': 'unit',
        'keyword': 'toner module, laser printer, colour',
        'factor': 1.14 # EI: It has a mass of 880 gr and an additional mass of 260 gr for the toner
    },
    'computer_laptop': {
        'unit': 'unit',
        'keyword': 'computer, laptop',
        'factor': 3.15 # EI:  and a total mass with expansion base 3.15 kg
    },
    'consumerelectronics_tablet': {
        'unit': 'unit',
        'keyword': 'consumer electronics, mobile device, tablet',
        'factor': 0.519 # EI:The scaling was done with an average composition of tablets resulting in a weight of 519g per unit.
    },
    'computer_desktop_wo_screen': {
        'unit': 'unit',
        'keyword': 'computer, desktop, without screen',
        'factor': 11.3 # EI: total weight without screen and cardboard packaging 11.3 kg
    },
    'keyboard': {
        'unit': 'unit',
        'keyword': 'keyboard',
        'factor': 1.18 # EI:  Its production represents a common keyboard with a mass of 1.18 kg and 102 keys
    },
    'pointingdevice': {
        'unit': 'unit',
        'keyword': 'pointing device, optical mouse, with cable',
        'factor': 0.12 # ei: total mass 0.120 kg
    },
    'flexible_duct_aluminimupet_dn125': {
        'unit': 'meter',
        'keyword': 'flexible duct, aluminium/pet, dn of 125',
        'factor': 0.205 # EI: Weight 0.205 kg/m
    },
    'printer_laser_blackwhite': {
        'unit': 'unit',
        'keyword': 'printer, laser, black/white',
        'factor': 4.61 # ei: with a total weight of 4.61 kg.
    },
    'printer_laser_colour': {
        'unit': 'unit',
        'keyword': 'printer, laser, colour',
        'factor': 4.61 # ei: with a total weight of 4.61 kg.
    },
    'harddisk_laptop': {
        'unit': 'unit',
        'keyword': 'hard disk drive, for laptop computer',
        'factor': 0.115 # ei: total weight of 0.115 kg
    },
    'harddisk': {
        'unit': 'unit',
        'keyword': 'hard disk drive, for desktop computer',
        'factor': 0.575 # ei: total weight 0.575 kg
    },
    'diskdrive_laptop': {
        'unit': 'unit',
        'keyword': 'disk drive, cd/dvd, rom, for laptop computer',
        'factor': 0.249 # ei:  total weight of 0.249 kg
    },
    'diskdrive_desktop': {
        'unit': 'unit',
        'keyword': 'disk drive, cd/dvd, rom, for desktop computer',
        'factor': 0.938 # ei:  total weight of 0.938 kg
    },
    'cable_printercableo_wo_plugs': {
        'unit': 'meter',
        'keyword': 'cable, printer cable, without plugs',
        'factor': 0.072 # ei: weight of 0.072 kg
    },
    'plug_io_computercable': {
        'unit': 'unit',
        'keyword': 'plug, inlet and outlet, for computer cable',
        'factor': 0.0585 # ei:  It is based on a inlet plug (with pins)with a weight of 0.0293 kg and 0.0292 kg for the outlet plug. 
    },
    'cable_data_infrastructure': {
        'unit': 'meter',
        'keyword': 'cable, data cable in infrastructure',
        'factor': 0.079 # ei: with a meter-weight of 0.03 resp. 0.079 kg/m
    },
    'cable_computer_wo_plugs': {
        'unit': 'meter',
        'keyword': 'cable, connector for computer, without plugs',
        'factor': 0.065 # ei: with a weight of 0.065 kg (without plugs)
    },
    'plug_io_printer': {
        'unit': 'unit',
        'keyword': 'plug, inlet and outlet, for printer cable',
        'factor': 0.0895 # ei: based on a inlet plug with a weight of 0.0395 kg and a outlet plug of 0.050 kg
    },
    'poweradaptor_laptop': {
        'unit': 'unit',
        'keyword': 'power adapter, for laptop',
        'factor': 0.531 #  EI: with a total mass of 0.357 kg and and additional mass of 0.174 kg for the belonging cables
    },
    'marine_electric_motor': {
        'unit': 'unit',
        'keyword': 'marine electric motor',
        'factor': 1000 # EI: This dataset represents the construction of electric motors and pumps used in marine applications, per 1000 kg of motor
    },
    'powersupply_unit_deskptop': {
        'unit': 'unit',
        'keyword': 'power supply unit, for desktop computer',
        'factor': 1.4434 # Estimated from EI material inputs 
    },
    'generator_minichp_plant': {
        'unit': 'unit',
        'keyword': 'generator, mini chp plant',
        'factor': 12 # Derived from EI material inputs
    },
    'plug_io_network_cable': {
        'unit': 'unit',
        'keyword': 'plug, inlet and outlet, for network cable',
        'factor': 0.0086 # EI: Its production represents a two plugs (2 identical ones) set, with a mass of 0.0086 kg
    },
    'cogen_unit_1mw_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 1mw electrical, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure 
    },
    'cogen_unit_6400kw_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 6400kw thermal, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure 
    },
    'generator_200kw': {
        'unit': 'unit',
        'keyword': 'generator, 200kw electrical',
        'factor': 0 # Exclude: this is infrastructure 
    },
    'poweradapter_smartphone': {
        'unit': 'unit',
        'keyword': 'power adapter, for smartphone',
        'factor': 0.0597 # EI: smartphone with a weight of 0.0597kg
    },
    'windturbine_connection_750kw': {
        'unit': 'unit',
        'keyword': 'wind turbine network connection, 750kw, onshore',
        'factor': 0 # Exclude: this is construction of infrastucture
    },
    'windturbine_connection_2mw': {
        'unit': 'unit',
        'keyword': 'wind turbine network connection, 2mw, onshore',
        'factor': 0 # Exclude: this is construction of infrastucture
    },
    'windturbine_connection_4_5mw': {
        'unit': 'unit',
        'keyword': 'wind turbine network connection, 4.5mw, onshore',
        'factor': 0 # Exclude: this is construction of infrastucture 
    },
    'windturbine_connection_6kw': {
        'unit': 'unit',
        'keyword': 'wind turbine network connection, small-scale, 6kw, onshore',
        'factor': 0 # Exclude: this is construction of infrastucture 
    },
    'inverter_500kw': {
        'unit': 'unit',
        'keyword': 'inverter, 500kw',
        'factor': 3000 # ei: Total weight about 3000 kg.
    },
    'inverter_2_5kw': {
        'unit': 'unit',
        'keyword': 'inverter, 2.5kw',
        'factor': 18.5 # ei: Total weight about 18.5 kg.
    },
    'inverter_0_5kw': {
        'unit': 'unit',
        'keyword': 'inverter, 0.5kw',
        'factor': 1.6 # EI: Total weight about 1.6 kg.
    },
    'cable_network_5_wo_plugs': {
        'unit': 'meter',
        'keyword': 'cable, network cable, category 5, without plugs',
        'factor': 0.036 # EI: with a mass of 0.036kg/m
    },
    'cogen_unit_200kw_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 200kw electrical, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'cogen_unit_200kw_diesel_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 200kw electrical, diesel scr, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'dieselelectric_set_10mw': {
        'unit': 'unit',
        'keyword': 'diesel-electric generating set, 10mw',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'electricparts_minichpplant': {
        'unit': 'unit',
        'keyword': 'electric parts, mini chp plant',
        'factor': 50 # EI: The control cabinet with the electrical equipment weighs around 50 kg
    },
    'electricparts_cogenunit_160kw': {
        'unit': 'unit',
        'keyword': 'electric parts, heat and power co-generation unit, 160kw electrical',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'minichpplant_componentselect': {
        'unit': 'unit',
        'keyword': 'mini chp plant, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'cogen_unit_50kw_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 50kw electrical, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure unit  
    },
    'dieselelectric_set_18_5mw': {
        'unit': 'unit',
        'keyword': 'diesel-electric generating set, 18.5kw',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'cogen_unit_160kw_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, 160kw electrical, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'cable_threeconductor': {
        'unit': 'meter',
        'keyword': 'cable, three-conductor cable',
        'factor': 1.04 # EI:  with a meter-weight of 1.04 kg/m
    },
    'fuelcell_so_mgturbine_180kw': {
        'unit': 'unit',
        'keyword': 'fuel cell, solid oxide, with micro gas turbine, 180kw electrical, future',
        'factor': 0 # Exclude 
    },
    'cogen_unit_orc_1400kw_componentselect': {
        'unit': 'unit',
        'keyword': 'heat and power co-generation unit, organic rankine cycle, 1400kw thermal, components for electricity only',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'fuelcell_stack_polymer_2kw': {
        'unit': 'unit',
        'keyword': 'fuel cell, stack polymer electrolyte membrane, 2kw electrical, future',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'fuelcell_polymer_2kw': {
        'unit': 'unit',
        'keyword': 'fuel cell, polymer electrolyte membrane, 2kw electrical, future',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'fuelcell_so_125kw': {
        'unit': 'unit',
        'keyword': 'fuel cell, solid oxide, 125kw electrical, future',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'fuelcell_stack_so_125kw': {
        'unit': 'unit',
        'keyword': 'fuel cell, stack solid oxide, 125kw electrical, future',
        'factor': 0 # Exclude: this is infrastructure unit 
    },
    'catalyticconverter_threeway_19l': {
        'unit': 'unit',
        'keyword': 'catalytic converter, three-way, 19.1l',
        'factor': 70 #  EI: The infrastructure has a capacity of 19.1 liters and weighs 70 kg
    },
    'catalyticconverter_selective_200l': {
        'unit': 'unit',
        'keyword': 'catalytic converter, selective catalytic reduction, 200 litre',
        'factor': 240.3 # Derived from EI material inputs
    },
    'catalyticconverter_oxidation_20l': {
        'unit': 'unit',
        'keyword': 'catalytic converter, oxidation, 20 litre',
        'factor': 74.03 # Derived from EI material inputs
    },
    'catalyticconverter_threeway_minichp': {
        'unit': 'unit',
        'keyword': 'catalytic converter, three-way, mini chp plant',
        'factor': 0.5 # EI: The volume of the catalyst used for the mini CHP was given as 0.27 liters, the weight as 0.5 kg
    },
    'printed_wiringboard_throughhole_pb': {
        'unit': 'square meter',
        'keyword': 'printed wiring board, for through-hole mounting, pb containing surface',
        'factor': 3.08 # ei: with a square weight of 3.08 kg
    },
    'printed_wiringboard_surfacemounting_pb': {
        'unit': 'square meter',
        'keyword': 'printed wiring board, for surface mounting, pb containing surface',
        'factor': 3.26 # EI: with a square weight of 3.26 kg
    },
    'printed_wiringboard_surfacemounting_pbfree': {
        'unit': 'square meter',
        'keyword': 'printed wiring board, for surface mounting, pb free surface',
        'factor': 3.26 # EI: with a square weight of 3.26 kg
    },
    'printed_wiringboard_throughhole_pbfree': {
        'unit': 'square meter',
        'keyword': 'printed wiring board, for through-hole mounting, pb free surface',
        'factor': 3.08 # ei: with a square weight of 3.08 kg
    },
    'pv_panel': {
        'unit': 'square meter',
        'keyword': 'photovoltaic panel',
        'factor': 0 # exclude | no weight given 
    },
    'pv_laminate': {
        'unit': 'square meter',
        'keyword': 'photovoltaic laminate',
        'factor': 0 # exclude | no weight given 
    },
    'pv_cell': {
        'unit': 'square meter',
        'keyword': 'photovoltaic cell',
        'factor': 0 # exclude | no weight given 
    },
    'printed_wiringboard_mountedmainboard_pbfree': {
        'unit': 'square meter',
        'keyword': 'printed wiring board, mounted mainboard, mobile device, double-sided, pb free',
        'factor': 1 # EI: This activity represents the production of 1kg of printed wiring board (PWB) for use in mobile devices
    },
    'multisi_wafer': {
        'unit': 'square meter',
        'keyword': 'multi-si wafer',
        'factor': 0.559 # EI: The weight is 559 g/m2.
    },
    'singlesi_wafer_elect': {
        'unit': 'square meter',
        'keyword': 'single-si wafer, for electronics',
        'factor': 0.629 # EI: The weight is 629 g/m2.
    },
    'multisi_wafer_ribbon': {
        'unit': 'square meter',
        'keyword': 'multi-si wafer, ribbon',
        'factor': 0.583 # EI: The weight is 583 g/m2
    },
    'singlesi_wafer_pv': {
        'unit': 'square meter',
        'keyword': 'single-si wafer, photovoltaic',
        'factor': 0.629 # EI: The weight is 629 g/m2.
    },
    'consumerelectronics_smartphone': {
        'unit': 'unit',
        'keyword': 'consumer electronics, mobile device, smartphone',
        'factor': 0.16345 # EI: a total weight of 163.45g
    },
    'internet_access_equipment': {
        'unit': 'unit',
        'keyword': 'internet access equipment',
        'factor': 0.29702 # Derived from EI material inputs 
    },
    'router_internet': {
        'unit': 'unit',
        'keyword': 'router, internet',
        'factor': 2.13765953 # Derived from EI material inputs 
    },
    'display_liquidcrystal_17': {
        'unit': 'unit',
        'keyword': 'display, liquid crystal, 17 inches',
        'factor': 11.95829701 # Derived from EI material inputs
    },
    'bus': {
        'unit': 'unit',
        'keyword': 'bus',
        'factor': 11000 # EI: Material consumption values refer to the production of one average lorry in the referring weight class (net weight 11000)
    },
    'light_commercial_vehicle': {
        'unit': 'unit',
        'keyword': 'light commercial vehicle',
        'factor': 2663.45744 # Derived from EI material inputs
    },
    'lorry_28': {
        'unit': 'unit',
        'keyword': 'lorry, 28 metric ton',
        'factor': 10494.22 # Estimated from EI material inputs
    },
    'lorry_16': {
        'unit': 'unit',
        'keyword': 'lorry, 16 metric ton',
        'factor': 6460.72 # Estimated from EI material inputs
    },
    'lorry_40': {
        'unit': 'unit',
        'keyword': 'lorry, 40 metric ton',
        'factor': 15420.42 # 
    },
    'wastecollection_lorry_21mton': {
        'unit': 'unit',
        'keyword': 'waste collection lorry, 21 metric ton',
        'factor': 8398.936 # Ecoinvent models it as lorry 16 metric ton x 1.3 | 6460.72*1.3
    },
    'silencer_steel_dn125': {
        'unit': 'unit',
        'keyword': 'silencer, steel, dn 125',
        'factor': 8.3 # the mass of each element is 8.3 kg
    },
    'display_cathoderay_17': {
        'unit': 'unit',
        'keyword': 'display, cathode ray tube, 17 inches',
        'factor': 25.9375305 # Estimated from EI material inputs 
    },
    'silencer_steel_dn315_50mm': {
        'unit': 'unit',
        'keyword': 'silencer, steel, dn 315, 50 mm',
        'factor': 20 # EI: Weight of element 20 kg
    },
    'intermodal_shipping_20foot': {
        'unit': 'unit',
        'keyword': 'intermodal shipping container, 20-foot',
        'factor': 2200 # EI: empty weight: 2200 kg
    },
    'intermodal_shipping_45foot_highcube': {
        'unit': 'unit',
        'keyword': 'intermodal shipping container, 45-foot, high-cube',
        'factor': 4800 # EI
    },
    'intermodal_shipping_40foot': {
        'unit': 'unit',
        'keyword': 'intermodal shipping container, 40-foot',
        'factor': 3800 # EI
    },
    'intermodal_shipping_40foot_highcube': {
        'unit': 'unit',
        'keyword': 'intermodal shipping container, 40-foot, high-cube',
        'factor': 3900 # EI
    },
    'refeer_intermodal_40foot_co2': {
        'unit': 'unit',
        'keyword': 'reefer, intermodal shipping container, 40-foot, high-cube, carbon dioxide, liquid as refrigerant',
        'factor': 3900 # 
    },
    'tanker_petroleum': {
        'unit': 'unit',
        'keyword': 'tanker, for petroleum',
        'factor': 7872663.643 # Derived from EI material inputs
    },
    'tanker_liquified_ng': {
        'unit': 'unit',
        'keyword': 'tanker, for liquefied natural gas',
        'factor': 8017924.63 # Derived from EI material inputs
    },
    'tanker_liquidgoods': {
        'unit': 'unit',
        'keyword': 'tanker, for liquid goods other than petroleum and liquefied natural gas',
        'factor': 19365693.15 # Derived from EI material inputs
    },
    'barge_tanker': {
        'unit': 'unit',
        'keyword': 'barge tanker',
        'factor': 359984 # Derived from EI material inputs
    },
    'ferry': {
        'unit': 'unit',
        'keyword': 'ferry',
        'factor': 3921592.08 # Derived from EI material inputs
    },
    'container_ship': {
        'unit': 'unit',
        'keyword': 'container ship',
        'factor': 19325794.65 # Derived from EI material inputs
    },
    'bulkcarrier_dry_goods': {
        'unit': 'unit',
        'keyword': 'bulk carrier, for dry goods',
        'factor': 11683100.41 # Derived from EI material inputs
    },
    'barge': {
        'unit': 'unit',
        'keyword': 'barge', 
        'factor': 300042 # Derived from EI material inputs
    },
    'offshore_platform_ng': {
        'unit': 'unit',
        'keyword': 'offshore platform, natural gas',
        'factor': 13464517.2 # Derived from EI material inputs
    },
    'offshore_platform_petroleum': {
        'unit': 'unit',
        'keyword': 'offshore platform, petroleum',
        'factor': 1291107.64 #  Derived from EI material inputs
    },
    'train_passenger_longdistance': {
        'unit': 'unit',
        'keyword': 'train, passenger, long-distance',
        'factor': 317000 # EI:  total weight of 317t
    },
    'train_passenger_highspeed': {
        'unit': 'unit',
        'keyword': 'train, passenger, high speed',
        'factor': 664800 # Derived from EI material inputs
    },
    'train_passenger_regional': {
        'unit': 'unit',
        'keyword': 'train, passenger, regional',
        'factor': 171000 # EI: Total weight of the train is about 171t
    },
    'tram': {
        'unit': 'unit',
        'keyword': 'tram',
        'factor': 20992.1 # Derived from EI material inputs
    },
    'goods_wagon': {
        'unit': 'unit',
        'keyword': 'goods wagon',
        'factor': 22218 # Derived from EI material inputs
    },
    'helicopter': {
        'unit': 'unit',
        'keyword': 'helicopter',
        'factor': 1000 # Derived from EI material inputs
    },
    'aircraft_passenger_longhaul': {
        'unit': 'unit',
        'keyword': 'aircraft, passenger, long haul',
        'factor': 134900 # EI: The aircraft has an operating empty weight (OEW) of 134.9 tons
    },
    'aircraft_belly_mediumhaul': {
        'unit': 'unit',
        'keyword': 'aircraft, belly freight, medium haul',
        'factor': 65300 # EI: The aircraft has an operating empty weight (OEW) of 65.3 tons.
    },
    'aircraft_passenger_veryshort': {
        'unit': 'unit',
        'keyword': 'aircraft, passenger, very short haul',
        'factor': 38300 # EI: The aircraft has an operating empty weight (OEW) of 38.3 tons 
    },
    'aircraft_passenger_mediumhaul': {
        'unit': 'unit',
        'keyword': 'aircraft, passenger, medium haul',
        'factor': 52400 # EI: The aircraft has an operating empty weight (OEW) of 52.4 tons
    },
    'aircraft_dedicatedfreight_veryshorthaul': {
        'unit': 'unit',
        'keyword': 'aircraft, dedicated freight, very short haul',
        'factor': 137768 # Estimated from EI material inputs
    },
    'aircraft_agricultural': {
        'unit': 'unit',
        'keyword': 'aircraft, agricultural',
        'factor': 2012 # EI: It weights 2012 kg.
    },
    'motor_scooter_50ccm': {
        'unit': 'unit',
        'keyword': 'motor scooter, 50 cubic cm engine',
        'factor': 112.2833971 # Estimated from EI material inputs 
    },
    'electric_bicycle': {
        'unit': 'unit',
        'keyword': 'electric bicycle',
        'factor': 24 # EI: The data set reflects a e-bike of 24kg including additionals (e.g. carriers, lights)
    },
    'bicycle': {
        'unit': 'unit',
        'keyword': 'bicycle',
        'factor': 17 # EI: The data set reflects a bike of 17kg including additionals (e.g. carriers, lights).
    },
    'aircraft_belly_longhaul': {
        'unit': 'unit',
        'keyword': 'aircraft, belly freight, long haul',
        'factor': 142200 # EI: The aircraft has an operating empty weight (OEW) of 142.2 tons and each passenger is accounted for 105 kg.
    },
    'aircraft_belly_veryshorthaul': {
        'unit': 'unit',
        'keyword': 'aircraft, belly freight, very short haul',
        'factor': 46600 # EI: The aircraft has an operating empty weight (OEW) of 46.6 tons.
    },
    'aircraft_dedicatedfreight_shorthaul': {
        'unit': 'unit',
        'keyword': 'aircraft, dedicated freight, short haul',
        'factor': 145000 # EI: The aircraft has an operating empty weight (OEW) of 145.0 tons
    },
    'aircraft_passenger_shorthaul': {
        'unit': 'unit',
        'keyword': 'aircraft, passenger, short haul',
        'factor': 43100 # EI: The aircraft has an operating empty weight (OEW) of 43.1 tons and each passenger is accounted for 105 kg.
    },
    'wafer_fabricated_integrated_circuit': {
        'unit': 'square meter',
        'keyword': 'wafer, fabricated, for integrated circuit',
        'factor': 0.7 # EI: It is assumed that 1m2 of wafer weighs 0.7kg.
    },
    'aircraft_belly_shorthaul': {
        'unit': 'unit',
        'keyword': 'aircraft, belly freight, short haul',
        'factor': 50500 # EI: The aircraft has an operating empty weight (OEW) of 50.5 tons.
    },
    'aircraft_dedicatedfreight_mediumhaul': {
        'unit': 'unit',
        'keyword': 'aircraft, dedicated freight, medium haul',
        'factor': 150600 # EI: The aircraft has an operating empty weight (OEW) of 150.6 tons
    },
    'aircraft_dedicatedfreight_longthaul': {
        'unit': 'unit',
        'keyword': 'aircraft, dedicated freight, long haul',
        'factor': 167700 # EI: The aircraft has an operating empty weight (OEW) of 167.7 tons
    },
}

In [84]:
final_df.head()

,CPC,HS,i,exporter_iso2,exporter,unit_value,mean_unit,min_unit,max_unit,v_sum
0,1171,100410,4,AF,Afghanistan,0.466782,0.466782,0.466782,0.466782,883.618
1,1930,121190,8,AL,Albania,4.626658,8.499452,0.932443,45.045455,48789.133
2,36410,630520,8,AL,Albania,11.197804,26.025642,9.306683,87.000000,15982.727
3,41113,720241,8,AL,Albania,2.713260,2.868185,1.477375,4.002427,149757.144
4,37430,252310,12,DZ,Algeria,0.064900,0.775937,0.012033,9.611111,346404.892


In [85]:
entries = []
handled_cpcs = set()
base_handled = set()
matched_activities = set()

for _, row in final_df.iterrows():
    cpc_code = int(row['CPC'])
    base_entry = {
        'supplier': {
            'classifications': {'CPC': [cpc_code]},
            'location': row['exporter_iso2'],
            'matrix': 'technosphere'
        },
        'unit_value': float(row['unit_value']),
        'min': float(row['min_unit']),
        'max': float(row['max_unit']),
        'mean': float(row['mean_unit']),
        'unit': 'kilogram',
        'weight':float(row['v_sum'])
    }
    entries.append(base_entry)
    base_handled.add(cpc_code)

    matched_any = False
    for entry in unit_info.get(cpc_code, []):
        for rule in conversion_rules.values():
            if rule['unit'] in entry['unit'] and rule['keyword'] in entry['reference product']:
                converted = base_entry.copy()
                converted['unit_value'] = base_entry['unit_value'] * rule['factor']
                converted['min'] = base_entry['min'] * rule['factor']
                converted['max'] = base_entry['max'] * rule['factor']
                converted['mean'] = base_entry['mean'] * rule['factor']
                converted['unit'] = rule['unit']
                converted['supplier'] = converted['supplier'].copy()
                # converted['supplier']['reference product'] = entry['reference product']
                converted['supplier']['location'] = entry['location']
                entries.append(converted)
                matched_activities.add(entry['key'])
                matched_any = True
    if matched_any:
        handled_cpcs.add(cpc_code)

In [86]:
# Summary of CPCs where we applied conversion rules
print("✅ CPCs with conversion rules applied:")
for c in sorted(handled_cpcs):
    print(f"- {c}")

# Build unmatched_per_cpc: only activities with non-kg units that no rule matched
unmatched_per_cpc = {}
for cpc_code, acts in unit_info.items():
    # select non-kg activities
    non_kg_acts = [a for a in acts if a['unit'] != 'kilogram']
    if not non_kg_acts:
        continue
    for a in non_kg_acts:
        matched = False
        for rule in conversion_rules.values():
            if rule['unit'] == a['unit'] and rule['keyword'] in a['reference product']:
                matched = True
                break
        if not matched:
            unmatched_per_cpc.setdefault(cpc_code, []).append(a)

# Print only CPCs with truly unmatched activities
print("⚠️  CPCs with non-kg units but no conversion rule:")
for cpc_code, acts in sorted(unmatched_per_cpc.items()):
    print(f"CPC {cpc_code}:")
    for a in acts:
        print(f"  - {a['name']} -> {a['reference product']} [{a['unit']}]" )

✅ CPCs with conversion rules applied:
⚠️  CPCs with non-kg units but no conversion rule:


# 8. JSON output

In [87]:
import copy
final_entries = copy.deepcopy(entries)

In [88]:
for entry in final_entries:
    entry["supplier"]["classifications"]["CPC"] = [str(cpc) for cpc in entry["supplier"]["classifications"]["CPC"]]
    #entry["supplier"]["unit"] = entry["unit"]
    entry["consumer"] = {
        "matrix": "technosphere",
    }
    entry["value"] = round(entry["unit_value"], 3)
    if entry["min"] != entry["max"]:
        entry["uncertainty"] = {
             "distribution": "triang",
             "parameters": {
                "loc": round(entry["mean"], 3),
                "minimum": round(entry["min"], 3),
                "maximum": round(entry["max"], 3),
             },
             "negative": 0
          }
    del entry["unit_value"]
    del entry["min"]
    del entry["max"]
    del entry["mean"]
    del entry["unit"]

final_entries = [
    f for f in final_entries
    if isinstance(f["supplier"]["location"], str)
]

In [89]:
with open('/Users/romain/GitHub/edges/edges/data/LCC 1.0_2023.json', 'w') as f:
    # json.dump(entries, f, indent=2)
    json.dump(
        {
            "name": "LCC method",
            "unit": "USD",
            "version": "1.0",
            "exchanges": final_entries,
        },
        f,
        separators=(',', ':'),
        ensure_ascii=False,
        indent=2
    )
print(f"Exported JSON config with {len(entries)} entries.")

Exported JSON config with 6506 entries.


In [108]:
for entry in final_entries:
    if not isinstance(entry["supplier"]["location"], str):
        print(entry)